In [ ]:
import numpy as np
import pandas as pd
import pcp
from matplotlib.colors import ListedColormap

def plot_decision_regions(X, y, classifier, resolution=0.01):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class examples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')


#dataf should have columns churn and prediction
def divide_frame(dataf,columns,random_state_sample = 1,random_state_ppn = 1,fraction = 0.7,eta = 0.1, iter = 50, plot = False):
    dataf_extract = dataf.loc[:,columns+["churn","prediction"]]
    dataf_extract_train = dataf_extract.sample(random_state = random_state_sample, frac = fraction)
    dataf_extract_test = dataf_extract.drop(dataf_extract_train.index)
    
    X = dataf_extract_train.loc[:,columns].values
    X_test = dataf_extract_test.loc[:,columns].values
    y = dataf_extract_train.loc[:,"churn"].values
    y_test = dataf_extract_test.loc[:,"churn"].values
    pnp = pcp.Perceptron(eta,iter,random_state_ppn)
    pnp.fit(X,y)

    if plot:
        plot_decision_regions(X_test, y_test, classifier=pnp)
        plt.xlabel('total day charge')
        plt.ylabel('total eve charge')
        plt.legend(loc='upper left')

    dataf_extract_test["prediction"] = pnp.predict(dataf_extract_test.loc[:,columns])
    errors = abs(dataf_extract_test["prediction"]-dataf_extract_test["churn"])/2

    print(errors.count(), " total in test, errors%: ",int(100*errors.sum()/errors.count()),"%")
    return pnp

def model(dataset):
    
    df = dataset[dataset["international plan"]==1]
    #nomads
    df[df["total intl calls"]<=2]["prediction"]=1#100% churn
    #regular callers
    cols = ["total intl charge","total day charge"]
    df[df["total intl calls"]>2]["prediction"]=intl_regular_ppn.predict(df[df["total intl calls"]>2].loc[:,cols])
    dataset[dataset["international plan"]==1]["prediction"] = df["prediction"]

    df = dataset[dataset["international plan"]==0]
    active = df[df['customer service calls']>=4] #Cusomer sevice calls >0 (267, 20) churn:  51.68539325842697 %
    tihoni_no = df[(df['customer service calls']<4) & (df['number vmail messages']==0)] #Customer service calls = 0, not using vmail (2216, 20) churn:  13.312274368231048 %
    tihoni_addicted = df[(df['customer service calls']<4) & (df['number vmail messages']>0)] 


In [ ]:
#prepare data
da = pd.read_csv("telecom_churn.csv")
da.drop(columns=['account length','phone number'],inplace = True)##
da['international plan'] = np.where(da['international plan']=='no', 0,1)
da['voice mail plan'] = np.where(da['voice mail plan']=='no', 0,1)

In [ ]:
prediction = model(da)
#check
errors = prediction ^ da["churn"]
percent = 100*errors.sum()/errors.count()
print(errors.count(),f"objects, total errors: {percent:.2f}%")